In [1]:
from esda.moran import Moran
import geopandas as gpd
import numpy as np
import pandas as pd
from libpysal.weights import Queen, KNN
from matplotlib import cm, colors
import matplotlib.pyplot as plt
from mgwr.gwr import GWR,MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import shift_colormap,truncate_colormap
from mpl_toolkits.axes_grid1 import make_axes_locatable
import multiprocessing as mp
from splot.esda import plot_moran
import statsmodels.api as sm

from statshelper import gwr_param_plots, q_q_plot
plt.style.use('dark_background')

In [2]:
df = pd.read_csv("./data/processed/Cleaned_Chicago_Sales.csv")
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
gdf = gdf.set_crs("EPSG:4326")
gdf = gdf.to_crs("EPSG:26916")
gdf = gdf.sample(frac=.01)

In [3]:
i_vars = [
    'Age',
    'Bedrooms',  
    'Building Square Feet', 
    'Estimate (Building)', 
    'Estimate (Land)', 
    'Fireplaces', 
    'Full Baths',
    'Garage 1 Area',
    'Garage 2 Area', 
    'Half Baths', 
    'Land Square Feet',
    'Rooms',
    'Sale Quarter'
    ]
dep_var = ['Sale Price']
other_to_keep = ["PIN", "Neighborhood Code", "Town Code", "geometry"]
my_rows = i_vars+dep_var+other_to_keep
gdf = gdf[my_rows]
print("Dataframe has {} rows and {} columns".format(len(gdf), len(list(gdf.columns))))
X = gdf[i_vars].values
y = gdf[dep_var].values
gdf.head()

Dataframe has 3243 rows and 18 columns


,Age,Bedrooms,Building Square Feet,Estimate (Building),Estimate (Land),Fireplaces,Full Baths,Garage 1 Area,Garage 2 Area,Half Baths,Land Square Feet,Rooms,Sale Quarter,Sale Price,PIN,Neighborhood Code,Town Code,geometry
63757,1.075117,-0.292681,-0.696047,0.679108,-0.170077,1.422378,-0.891279,0.55,-0.041762,-0.848209,-0.221088,-0.268846,-0.732717,0.661621,16182100240000,50,27,POINT (434377.816 4636459.347)
131828,1.520937,2.191616,0.932748,1.154365,1.155739,-0.599533,1.528825,0.55,-0.041762,-0.848209,-1.316099,1.734472,-1.483905,1.011756,14311040320000,170,77,POINT (443311.049 4641524.658)
154713,-1.790873,0.550809,0.624344,0.394783,-0.523800,-0.599533,0.635637,0.55,-0.041762,1.066654,-0.465473,-0.268846,-0.106728,-0.092429,25021170010000,121,70,POINT (450330.146 4620084.704)
85979,-0.357878,-0.292681,-0.454796,0.369117,0.091706,-0.599533,-0.891279,0.55,-0.041762,1.066654,0.456170,-0.268846,-0.607519,0.181711,9242130010000,80,22,POINT (432887.811 4654392.177)
43522,-0.899231,-0.292681,-0.321290,-0.467782,-0.620733,1.422378,-0.891279,0.55,-0.041762,1.066654,0.641257,-0.268846,1.020054,-0.656647,31221090130000,150,32,POINT (439295.729 4595867.416)


In [4]:
coords = [ np.array((geom.xy[0][0], geom.xy[1][0])) for geom in gdf['geometry'] ]

In [ ]:
mgwr_selector = Sel_BW(coords, y, X, multi=True)
mgwr_bw = mgwr_selector.search()
print("Selected optimal bandwidth is:", mgwr_bw)

/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=4.47803e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=1.33139e-19): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=1.2315e-19): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=9.2891e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=4.54597e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-p

/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=9.47054e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=3.3228e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=9.04826e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=3.33378e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=9.49909e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-

/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=2.77165e-19): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=2.56769e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=1.04729e-19): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=1.6266e-35): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=9.79749e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-

/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=1.1683e-35): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=4.22894e-36): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=5.50489e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=2.27731e-19): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=2.59569e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-

/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=4.65217e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=5.009e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=9.39299e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=5.0191e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=9.34671e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-pa

...

/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=2.04556e-19): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=1.19156e-19): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=3.83128e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=7.80114e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=1.28812e-19): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site

/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=1.46938e-19): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=6.35796e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=7.32596e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=7.28674e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=1.07617e-19): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site

/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=5.42419e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=5.69503e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=3.62096e-36): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=5.55722e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=2.02834e-19): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site

/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=6.37572e-37): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=6.32129e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=5.63667e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=6.22058e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=1.09063e-36): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site

/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=1.30616e-19): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=7.10346e-36): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=5.62072e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=7.08734e-36): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=6.20199e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/home/lagrange/.local/lib/python3.8/site

In [ ]:
mgwr_results = MGWR(coords, y, X, mgwr_selector).fit()

In [ ]:
mgwr_results.summary()

In [ ]:
plt.scatter(mgwr_results.y, mgwr_results.resid_response.reshape(-1))
plt.title("residuals vs. fitted plot")
plt.show()

In [ ]:
q_q_plot(mgwr_results.resid_response.reshape(-1))

In [ ]:
#w = DistanceBand.from_dataframe(gdf,threshold=800) # needed k large enough that it was connected
w = KNN.from_dataframe(gdf,k=64)
w.transform = 'R'

In [ ]:
mgwr_moran = Moran(mgwr_results.resid_response.reshape(-1), w, permutations=199)
plot_moran(mgwr_moran, zstandard=True, figsize=(10,4))
plt.show()

In [ ]:
gwr_param_plots(mgwr_results, gdf, names=["intercept"]+i_vars, subplots=(4,4), figsize=(20,30))

In [ ]:
gwr_param_plots(mgwr_results, gdf, names=["intercept"]+i_vars, subplots=(3,5), filter_t=True, figsize=(30,20))